# Facies classification using Machine Learning #
## LA Team Submission 3 ## 
### _[Lukas Mosser](https://at.linkedin.com/in/lukas-mosser-9948b32b/en), [Alfredo De la Fuente](https://pe.linkedin.com/in/alfredodelafuenteb)_ ####

In this python notebook we explore several facies classification models such as Deep Neural Networks, Boosting Trees and  Bayesian, taking into account spatial dependencies to outperform the prediction model proposed in the [prediction facies from wel logs challenge](https://github.com/seg/2016-ml-contest). 

## Problem Modeling
----

The dataset we will use comes from a class excercise from The University of Kansas on [Neural Networks and Fuzzy Systems](http://www.people.ku.edu/~gbohling/EECS833/).  This exercise is based on a consortium project to use machine learning techniques to create a reservoir model of the largest gas fields in North America, the Hugoton and Panoma Fields. For more info on the origin of the data, see [Bohling and Dubois (2003)](http://www.kgs.ku.edu/PRS/publication/2003/ofr2003-50.pdf) and [Dubois et al. (2007)](http://dx.doi.org/10.1016/j.cageo.2006.08.011). 

The dataset we will use is log data from nine wells that have been labeled with a facies type based on oberservation of core.  We will use this log data to train a classifier to predict facies types. 

This data is from the Council Grove gas reservoir in Southwest Kansas.  The Panoma Council Grove Field is predominantly a carbonate gas reservoir encompassing 2700 square miles in Southwestern Kansas.  This dataset is from nine wells (with 4149 examples), consisting of a set of seven predictor variables and a rock facies (class) for each example vector and validation (test) data (830 examples from two wells) having the same seven predictor variables in the feature vector.  Facies are based on examination of cores from nine wells taken vertically at half-foot intervals. Predictor variables include five from wireline log measurements and two geologic constraining variables that are derived from geologic knowledge. These are essentially continuous variables sampled at a half-foot sample rate. 

The seven predictor variables are:
* Five wire line log curves include [gamma ray](http://petrowiki.org/Gamma_ray_logs) (GR), [resistivity logging](http://petrowiki.org/Resistivity_and_spontaneous_%28SP%29_logging) (ILD_log10),
[photoelectric effect](http://www.glossary.oilfield.slb.com/en/Terms/p/photoelectric_effect.aspx) (PE), [neutron-density porosity difference and average neutron-density porosity](http://petrowiki.org/Neutron_porosity_logs) (DeltaPHI and PHIND). Note, some wells do not have PE.
* Two geologic constraining variables: nonmarine-marine indicator (NM_M) and relative position (RELPOS)

The nine discrete facies (classes of rocks) are: 
1. Nonmarine sandstone
2. Nonmarine coarse siltstone 
3. Nonmarine fine siltstone 
4. Marine siltstone and shale 
5. Mudstone (limestone)
6. Wackestone (limestone)
7. Dolomite
8. Packstone-grainstone (limestone)
9. Phylloid-algal bafflestone (limestone)

These facies aren't discrete, and gradually blend into one another. Some have neighboring facies that are rather close.  Mislabeling within these neighboring facies can be expected to occur.  The following table lists the facies, their abbreviated labels and their approximate neighbors.

Facies |Label| Adjacent Facies
:---: | :---: |:--:
1 |SS| 2
2 |CSiS| 1,3
3 |FSiS| 2
4 |SiSh| 5
5 |MS| 4,6
6 |WS| 5,7
7 |D| 6,8
8 |PS| 6,7,9
9 |BS| 7,8

## Data Preprocessing
---

Let's import all the libraries that will be particularly needed for the analysis.

In [61]:
%%sh
pip install pandas
pip install scikit-learn
pip install keras
pip install xgboost

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [62]:
from __future__ import print_function
import numpy as np
%matplotlib inline
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from classification_utilities import display_cm, display_adj_cm
from sklearn.metrics import confusion_matrix, f1_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

We load the training and testing data to preprocess it for further analysis.

In [63]:
filename = 'train_test_data.csv'
training_data = pd.read_csv(filename)
training_data.head(10)

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915
5,3,A1 SH,SHRIMPLIN,2795.5,73.97,0.636,14.0,13.385,3.6,1,0.894
6,3,A1 SH,SHRIMPLIN,2796.0,73.72,0.630,15.6,13.930,3.7,1,0.872
7,3,A1 SH,SHRIMPLIN,2796.5,75.65,0.625,16.5,13.920,3.5,1,0.830
8,3,A1 SH,SHRIMPLIN,2797.0,73.79,0.624,16.2,13.980,3.4,1,0.809
9,3,A1 SH,SHRIMPLIN,2797.5,76.89,0.615,16.9,14.220,3.5,1,0.787


We fill the missing data values in the PE field with zero and proceed to normalize the data that will be fed into our model.

In [64]:
# Set 'Well Name' and 'Formation' fields as categories
training_data['Well Name'] = training_data['Well Name'].astype('category')
training_data['Formation'] = training_data['Formation'].astype('category')

# Fill missing values and normalize for 'PE' field
training_data['PE'] = training_data['PE'].fillna(value=0)
mean_pe = training_data['PE'].mean()
std_pe = training_data['PE'].std()
training_data['PE'] = (training_data['PE']-mean_pe)/std_pe

# Normalize the rest of fields (GR, ILD_log10, DelthaPHI, PHIND,NM_M,RELPOS)
correct_facies_labels = training_data['Facies'].values
feature_vectors = training_data.drop(['Formation', 'Depth'], axis=1)
well_labels = training_data[['Well Name', 'Facies']].values
data_vectors = feature_vectors.drop(['Well Name', 'Facies'], axis=1).values

scaler = preprocessing.StandardScaler().fit(data_vectors)
scaled_features = scaler.transform(data_vectors)
data_out = np.hstack([well_labels, scaled_features])

In order to start training stage, it is required to format the data by considering a sliding window over the depth component in order to classify a given set of features at some specific depth for each well in the training set.

In [65]:
def preprocess(data_out):
    data = data_out
    well_data = {}
    well_names = list(set(data[:, 0]))
    for name in well_names:
        well_data[name] = [[], []]

    for row in data:
        well_data[row[0]][1].append(row[1])
        well_data[row[0]][0].append(list(row[2::]))

    # Sliding window
    positive_lag = 5
    negative_lag = 5

    chunks = []
    chunks_test = []
    chunk_length = positive_lag+negative_lag+1 
    chunks_facies = []
    for name in well_names:
        if name not in ['STUART', 'CRAWFORD']:
            test_well_data = well_data[name]
            log_values = np.array(test_well_data[0])
            log_values_padded = np.lib.pad(log_values, (negative_lag,positive_lag), 'edge')[:, negative_lag:-positive_lag]
            facies_values =  np.array(test_well_data[1])
            for i in range(log_values.shape[0]):
                chunks.append(log_values_padded[i:i+chunk_length, :])
                chunks_facies.append(facies_values[i])
        else:
            test_well_data = well_data[name]
            log_values = np.array(test_well_data[0])
            log_values_padded = np.lib.pad(log_values, (negative_lag,positive_lag), 'edge')[:, negative_lag:-positive_lag]
            facies_values =  np.array(test_well_data[1])
            for i in range(log_values.shape[0]):
                chunks_test.append(log_values_padded[i:i+chunk_length, :])
    
    chunks_facies = np.array(chunks_facies, dtype=np.int32)-1
    X_ = np.array(chunks)
    X = np.zeros((len(X_),len(X_[0][0]) * len(X_[0])))
    for i in range(len(X_)):
        X[i,:] = X_[i].flatten()
        
    X_test = np.array(chunks_test)
    X_test_out = np.zeros((len(X_test),len(X_test[0][0]) * len(X_test[0])))
    for i in range(len(X_test)):
        X_test_out[i,:] = X_test[i].flatten()
    y = np_utils.to_categorical(chunks_facies)
    return X, y, X_test_out

## Data Analysis
---

We will experiment with an ensemble of classification models to outperform the accuracy at predicting facies. As input we will consider a set of features extracted by padding a depth interval segment, that way we take into account spatial dependencies. As output we obtain a vector filled with values ranging from [0-8] that indicate the presence of each facie with respect to depth.

In [66]:
# Reproducibility
np.random.seed(1337) 
# Load data
X_train, y_train, X_test = preprocess(data_out)
# Obtain labels
y_labels = np.zeros((len(y_train),1))
for i in range(len(y_train)):
    y_labels[i] = np.argmax(y_train[i])

In order to evaluate our classification model accurary we will use the our following defined metrics, based on the confusion matrix once the classification is performed. The first metric only considers misclassification error and the second one takes into account the fact that facies could be misclassified if they belong to a same group with similar geological characteristics.

In [67]:
def accuracy(conf):
    total_correct = 0.
    nb_classes = conf.shape[0]
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
    acc = total_correct/sum(sum(conf))
    return acc

adjacent_facies = np.array([[1], [0,2], [1], [4], [3,5], [4,6,7], [5,7], [5,6,8], [6,7]])
facies_labels = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS','WS', 'D','PS', 'BS']

def accuracy_adjacent(conf, adjacent_facies):
    nb_classes = conf.shape[0]
    total_correct = 0.
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
        for j in adjacent_facies[i]:
            total_correct += conf[i][j]
    return total_correct / sum(sum(conf))

### Model 1 - Deep Neural Network 
Our model will be composed by a Deep Neural Network of an input layer, two hidden layers and an output layer.

In [68]:
# Set parameters
input_dim = 77
hidden_dim_1 = 128
hidden_dim_2 = 32
output_dim = 9 
batch_size = 32
nb_epoch = 50

In [69]:
def dnn_model():
    # Define the model
    model = Sequential()
    model.add(Dense(128, input_dim=77, init='normal', activation='relu'))
    model.add(Dense(32, input_dim=128, init='normal', activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(9, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Once the set of parameters are fixed, the training stage of our model begins. We perform a Cross Validation routine to evaluate the performance of the model.

In [70]:
estimator = KerasClassifier(build_fn=dnn_model, nb_epoch=50, batch_size=50, verbose=0)
# Cross Validation
kfold = KFold(n_splits=5, shuffle=True)
results_dnn = cross_val_score(estimator, X_train, y_train, cv=kfold)
print(' Cross Validation Results')
print( results_dnn )

 Cross Validation Results
[ 0.69759037  0.72530121  0.73253012  0.69036145  0.71411339]


In [71]:
# Load the model
model_dnn = dnn_model()
#Train model
model_dnn.fit(X_train, y_train, nb_epoch=50, verbose=0, shuffle = True)

In [72]:
# Predict Values on Training set
y_predicted = model_dnn.predict( X_train , batch_size=32, verbose=0)

# Print Report

# Format output [0 - 8 ]
y_ = np.zeros((len(y_train),1))
for i in range(len(y_train)):
    y_[i] = np.argmax(y_train[i])

y_predicted_ = np.zeros((len(y_predicted), 1))
for i in range(len(y_predicted)):
    y_predicted_[i] = np.argmax( y_predicted[i] )
    
# Confusion Matrix
conf = confusion_matrix(y_, y_predicted_)

# Print Results
print ("\nModel Report")
print ("-Accuracy: %.6f" % ( accuracy(conf) ))
print ("-Adjacent Accuracy: %.6f" % ( accuracy_adjacent(conf, adjacent_facies) ))
print ("\nConfusion Matrix")
display_cm(conf, facies_labels, display_metrics=True, hide_zeros=True)


Model Report
-Accuracy: 0.895638
-Adjacent Accuracy: 0.983128

Confusion Matrix
     Pred    SS  CSiS  FSiS  SiSh    MS    WS     D    PS    BS Total
     True
       SS   257    11                                             268
     CSiS    36   888    16                                       940
     FSiS     3    82   693                             2         780
     SiSh                 1   227     8    33     1     1         271
       MS                       1   204    77          14         296
       WS                 1     9    29   503     3    37         582
        D                                   3   130     8         141
       PS                             4    48     1   632     1   686
       BS                                   1           2   182   185

Precision  0.87  0.91  0.97  0.96  0.83  0.76  0.96  0.91  0.99  0.90
   Recall  0.96  0.94  0.89  0.84  0.69  0.86  0.92  0.92  0.98  0.90
       F1  0.91  0.92  0.93  0.89  0.75  0.81  0.94  0.91  0.99  0.9

### Model 2 - XGBoost


XGBoost (or Extreme Gradient Boosting) is a sophisticated algorithm that corresponds to an advanced implementation of gradient boosting algorithm. Despite it's complexity, it is fairly easy to use and very powerful to deal with many different types of irregularities in data. It has been no surprise then, that is has been extensively applied in many machine learning competitions to obtain very promising results.


Although XGBoost is a very straightforward algorithm to implement, it's difficulty arises in dealing with a big number of hyperparameters. For that reason, we need to develop routines to tune this parameters to optimize the algorithm performance on the data prediction.

We will use a Cross-Validation approach to improve our model by tuning parameters at each step. There are three types of parameters to consider:

- General Parameters: Guide the overall functioning
- Booster Parameters: Guide the individual booster (tree/regression) at each step
- Learning Task Parameters: Guide the optimization performed

We must realize that our training data is quite reduced, therefore to evaluate our model generalization

In [73]:
# Build Model
rounds = 100

model_xgb = xgb.XGBClassifier( learning_rate =0.001, n_estimators=500, max_depth=6,
                               min_child_weight=1, gamma=0, subsample=0.8, reg_alpha = 1,
                               colsample_bytree=0.8, objective='multi:softmax',
                               nthread=4, scale_pos_weight=1, seed=27)

# Cross Validation
kfold = KFold(n_splits=5, shuffle=True)
results_xgb = cross_val_score( model_xgb , X_train, y_labels, cv=kfold)
print(' Cross Validation Results')
print( results_xgb )


#Fit the algorithm on the data
model_xgb.fit(X_train, y_labels ,eval_metric='merror')

#Predict training set:
predictions = model_xgb.predict(X_train)
        
#Print model report

# Confusion Matrix
conf = confusion_matrix(y_labels, predictions )

# Confusion Matrix
print ("\nConfusion Matrix")
display_cm(conf, facies_labels, display_metrics=True, hide_zeros=True)

# Print Results
print ("\nModel Report")
print ("-Accuracy: %.6f" % ( accuracy(conf) ))
print ("-Adjacent Accuracy: %.6f" % ( accuracy_adjacent(conf, adjacent_facies) ))

 Cross Validation Results
[ 0.65421687  0.67349398  0.64578313  0.67590361  0.71773221]

Confusion Matrix
     Pred    SS  CSiS  FSiS  SiSh    MS    WS     D    PS    BS Total
     True
       SS   181    78     9                                       268
     CSiS    11   843    81     1           2           2         940
     FSiS         144   629     1           3           3         780
     SiSh           1     4   224     2    33     1     6         271
       MS           6     6     8   175    66     3    32         296
       WS                 2    22     5   485     5    63         582
        D                 1     4           7   113    16         141
       PS                 7     8          85     7   579         686
       BS                                   9          18   158   185

Precision  0.94  0.79  0.85  0.84  0.96  0.70  0.88  0.81  1.00  0.83
   Recall  0.68  0.90  0.81  0.83  0.59  0.83  0.80  0.84  0.85  0.82
       F1  0.79  0.84  0.83  0.83  0.73  0.

### Model 3 - Random Forrest



In [77]:
# Build Model
model_rfc = RandomForestClassifier(n_estimators=5, criterion='gini', 
                                   max_depth=None, min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                   max_features='auto', max_leaf_nodes=None, 
                                   min_impurity_split=1e-07, bootstrap=True, 
                                   oob_score=False, n_jobs=1, random_state=None, 
                                   verbose=0, warm_start=False, class_weight='balanced')

# Cross Validation
kfold = KFold(n_splits=5, shuffle=True)
results_rfc = cross_val_score( model_rfc , X_train, y_labels, cv=kfold)
print(' Cross Validation Results')
print( results_rfc )

#Fit the algorithm on the data
model_rfc.fit(X_train, y_labels)

#Predict training set:
predictions = model_rfc.predict(X_train)
        
#Print model report

# Confusion Matrix
conf = confusion_matrix(y_labels, predictions )

# Confusion Matrix
print ("\nConfusion Matrix")
display_cm(conf, facies_labels, display_metrics=True, hide_zeros=True)

# Print Results
print ("\nModel Report")
print ("-Accuracy: %.6f" % ( accuracy(conf) ))
print ("-Adjacent Accuracy: %.6f" % ( accuracy_adjacent(conf, adjacent_facies) ))

 Cross Validation Results
[ 0.71084337  0.68915663  0.68915663  0.70722892  0.68395657]

Confusion Matrix
     Pred    SS  CSiS  FSiS  SiSh    MS    WS     D    PS    BS Total
     True
       SS   260     6     2                                       268
     CSiS     4   924    11                 1                     940
     FSiS     3    19   757                             1         780
     SiSh           1         268     1     1                     271
       MS           1     2         286     3     1     3         296
       WS                 2     8     3   562           7         582
        D                                   1   140               141
       PS                 1     2     4    17     1   661         686
       BS           1                       4           1   179   185

Precision  0.97  0.97  0.98  0.96  0.97  0.95  0.99  0.98  1.00  0.97
   Recall  0.97  0.98  0.97  0.99  0.97  0.97  0.99  0.96  0.97  0.97
       F1  0.97  0.98  0.97  0.98  0.97  0.

/home/alfredo/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


## Prediction
---
We obtain the predictions for test data.

In [79]:
# DNN model Prediction
y_test = model_dnn.predict( X_test , batch_size=32, verbose=0)
predictions_dnn = np.zeros((len(y_test),1))
for i in range(len(y_test)):
    predictions_dnn[i] = np.argmax(y_test[i]) + 1 
# Store results
test_data = pd.read_csv('../validation_data_nofacies.csv')
test_data['Facies'] = predictions_dnn
test_data.to_csv('Prediction_DNN.csv')

In [81]:
# XGBoost model Prediction
predictions_xgb = model_xgb.predict(X_test) + 1
# Store results
test_data = pd.read_csv('../validation_data_nofacies.csv')
test_data['Facies'] = predictions_xgb
test_data.to_csv('Prediction_XGB.csv')

In [87]:
# Random Forrest
predictions_rfc = model_rfc.predict(X_test) + 1
# Store results
test_data = pd.read_csv('../validation_data_nofacies.csv')
test_data['Facies'] = predictions_rfc
test_data.to_csv('Prediction_RFC.csv')

In [116]:
# Embedded model

# Weights
coef_dnn = results_dnn.mean()
coef_xgb = results_xgb.mean()
coef_rfc = results_rfc.mean()

# Weighted Average
predictions_final = np.zeros((len(X_test),1))
for i in range( len(X_test ) ):
    predictions_final[i] = ( int ) ( round ( ( coef_dnn * predictions_dnn[i] +
                                       coef_xgb * predictions_xgb[i] +
                                       coef_rfc * predictions_rfc[i] ) / 
                                     ( coef_dnn + coef_xgb + coef_rfc ) )  )
    
# Store results
test_data = pd.read_csv('../validation_data_nofacies.csv')
test_data['Facies'] = predictions_final
test_data.to_csv('Prediction_FINAL.csv')